# Strategic Research Forecasting Report (2025-2035)

## Philippine Higher Education Institution Research Productivity Analysis

**Prepared for:** HEI Stakeholders & Academic Leadership

**Date:** January 2026

---

### Abstract

This report presents a comprehensive analysis of research productivity trends across Philippine Higher Education Institutions (HEIs) from 2015 to 2025, with forecasted projections through 2035. The analysis accounts for the structural disruption caused by the COVID-19 pandemic (2020-2022) and employs momentum-based forecasting to project future research output.

**Key Findings:**
- Research productivity experienced significant disruption during the pandemic period
- Post-pandemic recovery trends indicate resilient institutional capacity
- Regional disparities in research output remain significant, with NCR leading
- Forecasts project continued growth through 2035

## 1. Methodology

### 1.1 Pandemic Structural Break

The COVID-19 pandemic (2020-2022) introduced a **structural break** in research productivity data. This period saw:
- Laboratory closures and research delays
- Shift to remote work affecting collaborative research
- Publication pipeline disruptions

We treat 2020-2022 as a distinct period to avoid contaminating long-term trend estimates with pandemic-induced volatility.

### 1.2 Forecasting Approach: Holt's Linear Trend Method

We employ **Holt's Linear Trend (Double Exponential Smoothing)** for forecasting, which captures both level and trend components:

$$\hat{Y}_{t+h} = L_t + h \cdot T_t$$

Where:
- $\hat{Y}_{t+h}$ = Forecasted value at horizon $h$
- $L_t$ = Estimated level at time $t$
- $T_t$ = Estimated trend at time $t$
- $h$ = Forecast horizon (years ahead)

**Why Holt's Method?**
- **Momentum-based:** Prioritizes recent growth trajectory for expanding institutions
- **Smoothing:** Reduces noise from year-to-year fluctuations
- **Interpretable:** Clear level + trend decomposition

For institutions with sparse data (< 3 historical points), we fallback to **Simple Moving Average (SMA)** to provide conservative estimates.

### 1.3 Period Definitions

| Period | Years | Duration | Type |
|--------|-------|----------|------|
| Pre-Pandemic | 2015-2019 | 5 years | Historical |
| During Pandemic | 2020-2022 | 3 years | Historical |
| Post-Pandemic | 2023-2025 | 3 years | Historical |
| Forecast Phase 1 | 2026-2030 | 5 years | Forecast |
| Forecast Phase 2 | 2031-2035 | 5 years | Forecast |

## 2. National Overview

In [1]:
# Setup and Data Loading
import sys
sys.path.insert(0, '..')  # Add parent directory for src imports

import pandas as pd
import numpy as np

# Load forecast data
df = pd.read_parquet('../data/processed/forecasts.parquet')

# Period assignment function
def assign_period(year):
    if 2015 <= year <= 2019:
        return "Pre-Pandemic (2015-2019)"
    elif 2020 <= year <= 2022:
        return "During Pandemic (2020-2022)"
    elif 2023 <= year <= 2025:
        return "Post-Pandemic (2023-2025)"
    elif 2026 <= year <= 2030:
        return "Forecast Phase 1 (2026-2030)"
    elif 2031 <= year <= 2035:
        return "Forecast Phase 2 (2031-2035)"
    return "Unknown"

df['Period'] = df['Year'].apply(assign_period)

print(f"Dataset: {len(df):,} records")
print(f"Schools: {df['School'].nunique()}")
print(f"Regions: {df['Region'].nunique()}")
print(f"Year Range: {df['Year'].min()} - {df['Year'].max()}")

Dataset: 3,276 records
Schools: 52
Regions: 17
Year Range: 2015 - 2035


In [2]:
# National Summary by Period
period_order = [
    "Pre-Pandemic (2015-2019)",
    "During Pandemic (2020-2022)",
    "Post-Pandemic (2023-2025)",
    "Forecast Phase 1 (2026-2030)",
    "Forecast Phase 2 (2031-2035)"
]

# Calculate totals per period per metric
national_summary = df.pivot_table(
    index='Period',
    columns='Metric',
    values='Value',
    aggfunc='sum',
    observed=False
).reindex(period_order)

# Style the table
styled_summary = national_summary.style.format("{:,.0f}").background_gradient(
    cmap='YlGnBu', axis=0
).set_caption("National Research Output by Period")

styled_summary

Metric,Citation Quantity,Field-Weighted Citation Impact,Publication Quantity
Period,,,
Pre-Pandemic (2015-2019),"25,162",125,"2,278"
During Pandemic (2020-2022),"33,922",128,"3,649"
Post-Pandemic (2023-2025),"16,078",138,"6,766"
Forecast Phase 1 (2026-2030),"32,322",376,"19,652"
Forecast Phase 2 (2031-2035),"37,854",547,"30,390"


In [3]:
# Average Annual Values (for fair period comparison)
period_lengths = {
    "Pre-Pandemic (2015-2019)": 5,
    "During Pandemic (2020-2022)": 3,
    "Post-Pandemic (2023-2025)": 3,
    "Forecast Phase 1 (2026-2030)": 5,
    "Forecast Phase 2 (2031-2035)": 5
}

avg_annual = national_summary.copy()
for period in period_order:
    if period in avg_annual.index:
        avg_annual.loc[period] = avg_annual.loc[period] / period_lengths[period]

styled_avg = avg_annual.style.format("{:,.1f}").background_gradient(
    cmap='RdYlGn', axis=0
).set_caption("Average Annual Research Output by Period")

styled_avg

Metric,Citation Quantity,Field-Weighted Citation Impact,Publication Quantity
Period,,,
Pre-Pandemic (2015-2019),"5,032.4",25.1,455.6
During Pandemic (2020-2022),"11,307.3",42.7,"1,216.3"
Post-Pandemic (2023-2025),"5,359.3",46.0,"2,255.3"
Forecast Phase 1 (2026-2030),"6,464.4",75.1,"3,930.4"
Forecast Phase 2 (2031-2035),"7,570.8",109.4,"6,078.0"


## 3. Geospatial Analysis

The following animated map shows the regional evolution of research productivity across the 5 strategic periods. The color intensity represents the **average annual value** per region, ensuring fair comparison between periods of different lengths.

In [4]:
# Import visualization utilities
from src.viz_utils import plot_period_geospatial_comparison

# Generate animated map for Publication Quantity
fig_pub = plot_period_geospatial_comparison(df, "Publication Quantity")
fig_pub.show()

In [5]:
# Citation Quantity evolution
fig_cite = plot_period_geospatial_comparison(df, "Citation Quantity")
fig_cite.show()

In [6]:
# Regional breakdown table
regional_period = df[df['Metric'] == 'Publication Quantity'].pivot_table(
    index='Region',
    columns='Period',
    values='Value',
    aggfunc='sum',
    observed=False
)[period_order]

regional_period.style.format("{:,.0f}").background_gradient(
    cmap='Blues', axis=1
).set_caption("Publication Quantity by Region and Period")

Period,Pre-Pandemic (2015-2019),During Pandemic (2020-2022),Post-Pandemic (2023-2025),Forecast Phase 1 (2026-2030),Forecast Phase 2 (2031-2035)
Region,,,,,
BARMM,14,33,210,"1,075","2,000"
BICOL REGION,73,127,244,551,761
CAGAYAN VALLEY,212,274,304,766,"1,020"
CALABARZON,153,265,532,"1,230","1,688"
CAR,62,110,252,563,794
CARAGA REGION,141,202,474,"1,144","1,502"
CENTRAL LUZON REGION 3,331,591,"1,030","3,378","5,454"
CENTRAL VISAYAS,130,356,603,"1,462","2,066"
DAVAO REGION,35,53,139,602,"1,083"


## 4. Strategic Outlook (2026-2035)

### Key Projections

Based on the momentum-based Holt's Linear Trend forecasting:

1. **Continued Growth:** National research output is projected to grow steadily through 2035
2. **Regional Leaders:** NCR, CALABARZON, and Central Visayas maintain their leading positions
3. **Emerging Regions:** Several regions show accelerating growth trajectories

### Recommendations for HEI Stakeholders

1. **Investment in Research Infrastructure:** Sustained investment is needed to maintain growth trajectories
2. **Regional Collaboration:** Cross-regional research partnerships can help distribute research capacity
3. **Data-Driven Planning:** Use these forecasts for strategic resource allocation

---

*This report was generated using the Institutional Research Analytics Platform (IRAP). Forecasts are based on historical data and should be interpreted as projections subject to uncertainty.*

In [7]:
# Export data to Excel
from pathlib import Path

output_path = Path('HEI_Research_Report_Data.xlsx')

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    # National Summary
    national_summary.to_excel(writer, sheet_name='National Summary')
    
    # Regional Breakdown by Period
    for metric in df['Metric'].unique():
        metric_df = df[df['Metric'] == metric].pivot_table(
            index='Region',
            columns='Period',
            values='Value',
            aggfunc='sum'
        )
        # Clean sheet name (max 31 chars)
        sheet_name = f"Regional_{metric[:15]}"
        metric_df.to_excel(writer, sheet_name=sheet_name)
    
    # School Details (full data)
    school_details = df.pivot_table(
        index=['Region', 'School'],
        columns=['Metric', 'Year'],
        values='Value',
        aggfunc='first',
    observed=False
    )
    school_details.to_excel(writer, sheet_name='School Details')
    
    # Period Definitions reference
    period_ref = pd.DataFrame([
        {'Period': 'Pre-Pandemic', 'Years': '2015-2019', 'Duration': '5 years', 'Type': 'Historical'},
        {'Period': 'During Pandemic', 'Years': '2020-2022', 'Duration': '3 years', 'Type': 'Historical'},
        {'Period': 'Post-Pandemic', 'Years': '2023-2025', 'Duration': '3 years', 'Type': 'Historical'},
        {'Period': 'Forecast Phase 1', 'Years': '2026-2030', 'Duration': '5 years', 'Type': 'Forecast'},
        {'Period': 'Forecast Phase 2', 'Years': '2031-2035', 'Duration': '5 years', 'Type': 'Forecast'},
    ])
    period_ref.to_excel(writer, sheet_name='Period Definitions', index=False)

print(f"✅ Excel report exported to: {output_path.absolute()}")

✅ Excel report exported to: c:\Users\workj\PROG\publication-forecast-project\reports\HEI_Research_Report_Data.xlsx


C:\Users\workj\AppData\Local\Temp\ipykernel_39840\171788689.py:12: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior

C:\Users\workj\AppData\Local\Temp\ipykernel_39840\171788689.py:12: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior

C:\Users\workj\AppData\Local\Temp\ipykernel_39840\171788689.py:12: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior

